# Performance Comparison of Using Python Multi-threading and Multi-Processing

Multi-Processing & Multi-Threading 의 개념은 다음 링크에 정리 -> [Multi-Processing & Multi-Threading 의 개념](https://medium.com/@jihoju96/%EB%A9%80%ED%8B%B0-%ED%94%84%EB%A1%9C%EC%84%B8%EC%8B%B1-multi-processing-%EA%B3%BC-%EB%A9%80%ED%8B%B0-%EC%8A%A4%EB%A0%88%EB%94%A9-multi-threading-b981bb706f4b)

I/O bound 작업과 CPU bound 작업에 대해 Sing-thread/process, Multi-threading, Multi-Processing 프로그래밍 방식에 장단점이 존재한다. <br>
=> 이를 실습 예제를 통해 비교해보자.

- **IO-bound**: 프로세스가 진행되는 속도가 I/O 하위 시스템의 속도에 의해 제한됨. -> 파일의 행 수를 계산하는 것과 같이 디스크에서 데이터를 처리하는 작업
- **CPU-bound**: 프로세스 진행 속도가 CPU 속도에 의해 제한됨. -> 작은 숫자를 곱하는 것과 같이 작은 숫자 집합에서 계산을 수행하는 작업

### urllib.request

- **urllib**: URL 처리에 관련된 모듈을 모아 놓은 패키지
- **request**: urllib 패키지의 모듈 중 하나
- **urlopen**: URL(Uniform Resource Locator)을 가져오기 위한 함수로 URL 열기에 성공하면 response.status 값이 200이 나온다. 이 200 값은 HTTP 상태 코드이며 웹 서버가 요청을 제대로 처리했다는 뜻이다.

### requests vs urllib.request 차이점
### 1. requests

- 데이터를 보낼 때 딕셔너리 형태로 보낸다.
- 없는 페이지를 요청해도 에러를 띄우지 않고 <페이지를 띄울 수 없습니다> 같은 해당 내용을 응답으로 받는다.

### 2. urllib.request

- 데이터를 보낼 때 인코딩하여 바이너리 형태로 보낸다.
- 없는 페이지를 요청하면 에러를 띄운다.

=> 대체로 requests 를 많이 사용하며, request.get() 방법이 HTTP method와 연관되어 직관적이라 이해가 쉬운 거 같다.

# 1. I/O-bound Task

### 다음 예제는 urllib.request.urlopen 으로 list 에 담긴 여러 url 을 여는 IO-bound 작업이다.

### Single-thread/process

In [16]:
import urllib.request

In [17]:
urls = [
    'http://www.python.org',
    'https://docs.python.org/3/',
    'https://docs.python.org/3/whatsnew/3.7.html',
    'https://docs.python.org/3/tutorial/index.html',
    'https://docs.python.org/3/library/index.html',
    'https://docs.python.org/3/reference/index.html',
    'https://docs.python.org/3/using/index.html',
    'https://docs.python.org/3/howto/index.html',
    'https://docs.python.org/3/installing/index.html',
    'https://docs.python.org/3/distributing/index.html',
    'https://docs.python.org/3/extending/index.html',
    'https://docs.python.org/3/c-api/index.html',
    'https://docs.python.org/3/faq/index.html'
    ]

In [21]:
%%time

results = []
for url in urls:
    with urllib.request.urlopen(url) as src:
        results.append(src)

CPU times: user 269 ms, sys: 23.3 ms, total: 292 ms
Wall time: 740 ms


### Multi-threading

In [22]:
import urllib.request
from concurrent.futures import ThreadPoolExecutor

#### - map() 함수
You can also submit tasks by calling the map() function and specifying the name of the function to execute and the iterable of items to which your function will be applied.

In [23]:
%%time

with ThreadPoolExecutor(4) as executor:
    results = executor.map(urllib.request.urlopen, urls)

CPU times: user 115 ms, sys: 11.6 ms, total: 127 ms
Wall time: 210 ms


In [24]:
%%time

with ThreadPoolExecutor(8) as executor:
    results = executor.map(urllib.request.urlopen, urls)

CPU times: user 97.1 ms, sys: 10.6 ms, total: 108 ms
Wall time: 165 ms


In [25]:
%%time

with ThreadPoolExecutor(16) as executor:
    results = executor.map(urllib.request.urlopen, urls)

CPU times: user 89.4 ms, sys: 11.3 ms, total: 101 ms
Wall time: 149 ms


**단일 스레드의 경우 740ms** 가 소요되었지만, **멀티 스레드(4개)의 경우 210ms** 가 소요되었다.<br>
스레드를 1개에서 4개로 늘린 **멀티 스레드 환경에서 소요되는 시간이 71.62%가 감소했다.**

이처럼 **IO-bound 작업에서 Multi-Threading 은 비약적인 속도 향상**을 한다.

I/O bound 프로그램 수행 시, 컴퓨터 자원은 대부분의 시간을 input/output 을 기다리는데 사용한다. 

- Ex) network, database, file, user, etc...

I/O 작업을 여러 스레드가 동시에(concurrently) 수행하면 컴퓨터 자원(CPU)은 I/O 작업을 계속 기다리지 않고 다른 스레드를 수행할 수 있다.

### => **I/O bound 작업 시, Multi-threading은 성능을 비약적으로 향상**시킬 수 있다.

### **🙌 주의할 점**: Multi-Threaded는 I/O bound 작업 시 효율을 내지만 **CPU-bound 작업은 오히려 성능을 떨어뜨린다!!**

Python에서 Thread 는 **GIL(Global Interpreter Lock)에 종속**된다. 

- GIL은 참조 파이썬 인터프리터내의 프로그래밍 패턴이다.
- GIL은 lock 으로써 **한 프로세스 내의 한 명령어는 같은 시간에 하나의 스레드만이 수행할 수 있도록 한 동기화 기법**이다.

=> **Multi-Threading은 CPU 작업 + thread switching 이 수행되기 때문에 순차적인 처리보다 오히려 성능이 떨어진다.**

# 2. CPU-bound Task

### 다음 예제는 0 ~ 1000000 까지 모든 소수의 합을 구하는 연산 작업으로 CPU-bound 작업이다.

In [30]:
def is_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

### 단일 프로세스 및 스레드

In [33]:
%%time

answer = 0

for i in range(1000000):
    answer += is_prime(i)

CPU times: user 3.11 s, sys: 6.35 ms, total: 3.11 s
Wall time: 3.11 s


### Multi-threading

In [29]:
from concurrent.futures import ThreadPoolExecutor

In [32]:
%%time

with ThreadPoolExecutor(4) as executor:
    results = executor.map(is_prime, list(range(1000000)))

CPU times: user 11.9 s, sys: 294 ms, total: 12.2 s
Wall time: 12.1 s


위의 단일 스레드와 멀티 스레드 환경에서의 수행 시간을 비교해보면 Multi-threading은 오히려 성능을 매우 나쁘게 만든다.

- **단일 스레드: 3.11s**
- **멀티 스레드(4): 12.1s**
- **약 4배 차이**가 난다,,,

**=> CPU-bound 작업에 Multi-threading 방식을 사용하면 위의 결과처럼 성능을 더욱 악화시킨다.**

### Multi-Processing

In [28]:
from multiprocessing import Pool

In [34]:
%%time

# IPython 환경에서 같은 파일에 구현된 함수를 Multi-Processing 에 활용하면 의도치 않은 에러가 발생
# 위의 is_prime 함수를 다른 모듈에 구현 후 import
from practice_module import is_prime

if __name__ == '__main__':
    with Pool(2) as p:
        answer = sum(p.map(is_prime, list(range(1000000))))

CPU times: user 64.9 ms, sys: 58.5 ms, total: 123 ms
Wall time: 1.79 s


In [35]:
%%time

from practice_module import is_prime

if __name__ == '__main__':
    with Pool(4) as p:
        answer = sum(p.map(is_prime, list(range(1000000))))

CPU times: user 72 ms, sys: 51.7 ms, total: 124 ms
Wall time: 1 s


In [37]:
%%time

from practice_module import is_prime

if __name__ == '__main__':
    with Pool(8) as p:
        answer = sum(p.map(is_prime, list(range(1000000))))

CPU times: user 106 ms, sys: 82.5 ms, total: 189 ms
Wall time: 763 ms


In [38]:
%%time

from practice_module import is_prime

if __name__ == '__main__':
    with Pool(16) as p:
        answer = sum(p.map(is_prime, list(range(1000000))))

CPU times: user 119 ms, sys: 136 ms, total: 255 ms
Wall time: 822 ms


In [39]:
%%time

from practice_module import is_prime

if __name__ == '__main__':
    with Pool(32) as p:
        answer = sum(p.map(is_prime, list(range(1000000))))

CPU times: user 143 ms, sys: 247 ms, total: 389 ms
Wall time: 954 ms


In [40]:
%%time

from practice_module import is_prime

if __name__ == '__main__':
    with Pool(64) as p:
        answer = sum(p.map(is_prime, list(range(1000000))))

CPU times: user 195 ms, sys: 480 ms, total: 675 ms
Wall time: 1.29 s


- 이 작업을 수행하는 컴퓨터의 CPU core는 8개이다. 
- **8개의 멀티-프로세스 환경에서의 수행 속도가 가장 빨랐다.**
- **16개, 32개, 64개로 프로세스의 개수를 늘려갔을 때, 수행 시간이 점점 길어지는 것을 확인할 수 있다.**

CPU 보다 많은 프로세스들이 서로 **context-switching** 이 일어나며 속도가 줄어들어 위와 같은 결과가 나온 것이다.

**🙌 주의할 점 => Multi-Processing 시 CPU 개수 이상으로 프로세스를 생성하지 말자!**

위의 결과를 보면, multi-processing 방식은 CPU bound 작업의 수행 속도를 향상시켜주는 것을 알 수 있다.

- **단일 프로세스: 3.11s**
- **멀티 스레드(4): 12.1s**
- **멀티 프로세스(8): 763ms**

CPU bound 프로그램에서 컴퓨터 자원(CPU)은 대부분의 시간을 계산 연산 작업을 수행하는데 보낸다.

- ex) mathematical computations, image processing, etc. 

### Multi-prossing은 여러 프로세스가 독립적으로 **"동시에 or 같은 시간에(in parallel)"** 연산 작업을 수행하여 수행 속도를 향상시킬 수 있다.

다음 프로세스로 Mult-Processing 방식을 사용

1. **수행할 함수**를 정의
2. 정의한 함수를 적용시킬 리스트를 준비
3. **multiprocessing.Pool을 사용해서 작업을 수행할 프로세스를 생성**

    - Pool()의 인자로 들어가는 숫자는 생성할 프로세스의 개수이다.
    - with 으로 묶으면 작업이 끝난 후에 생성한 프로세스들이 종료된다. -> 효율적
    
4. **Pool의 메서드인 map 함수**를 사용하여 수행 결과를 합치자.

    - map 함수는 리스트 내 각 원소에 적용되는 함수이다.
    
### 🙌 주의할 점: I/O bound 작업 수행 시 multi-processing 으로 속도를 향상시킬 수 있지만 많은 프로세스를 관리하는데 오버헤드가 발생

- 각 프로세스마다 interpreter를 생성
- 별도의 독립적인 메모리 공간을 할당

이러한 이유로 **I/O bound 작업을 수행할 때는 오버헤드가 비교적 적은 multi-threading을 사용하는 것이 효율적**이다.

# 3. 정리

#### - I/O-bound 작업 시, multithreading 을 사용하면 성능이 향상된다.
#### - I/O-bound 작업 시, multiprocessing 도 역시 성능이 향상되지만, multithreading 을 사용하는 것보다 많은 오버헤드가 발생한다.
#### - Python GIL 은 파이썬 프로그램 내에서 같은 시간에 오직 한 스레드만이 수행될 수 있는 것을 의미한다.
#### - CPU-bound 작업 시, multithreading 을 사용하면 성능이 더욱 나빠진다.
#### - CPU-bound 작업 시, multiprocessing 을 사용하면 성능을 향상시킬 수 있다.

## References
- https://pub.towardsai.net/the-why-when-and-how-of-using-python-multi-threading-and-multi-processing-afd1b8a8ecca
- https://superfastpython.com/threadpoolexecutor-vs-threads/